# AI Community @ Семинар № 7, весна 2019
## State-of-the-art в компьютерном зрении. MobileNet, DenseNet, ResNeXt

## MobileNet [2017, CVPR]

[MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications](https://arxiv.org/abs/1704.04861)

### Ключевые особенности

1. Эффективная архитектура нейронной сети
1. Набор из двух гиперпараметров, которые позволяют построить сеть под конкретные условия (задача, порог точности, железо)

### Дизайн

#### Depthwise separable convolution

![Depthwise separable convolution](images/mobile_depthwise_conv.png)

[Объяснение](https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d) depthwise separable convolutions на примере

Стоимость обычной свертки: $D_K \cdot D_K \cdot M \cdot N \cdot D_F \cdot D_F$  
Стоимость depthwise separable convolution: $D_K \cdot D_K \cdot M \cdot D_F \cdot D_F + M \cdot N \cdot D_F \cdot D_F$  
Уменьшение стоимости: $\frac{1}{N} + \frac{1}{D^2_K}$

#### Архитектура

![Архитектура MobileNet](images/mobile_architecture.png)

#### Width Multiplier

Мы можем уменьшать стоимость вычислений, добавив параметр сжатия сверток $\alpha \in (0, 1]$. Тогда сложность вычислений depthwise separable convolution будет $D_K \cdot D_K \cdot \alpha M \cdot D_F \cdot D_F + \alpha M \cdot \alpha N \cdot D_F \cdot D_F$  
При этом сложность уменьшается примерно в $\alpha^2$

#### Resolution Multiplier

Также мы можем уменьшать размерность входных карт. Для этого используется параметр $\rho$. Результирующая сложность представлена $D_K \cdot D_K \cdot \alpha M \cdot \rho D_F \cdot \rho D_F + \alpha M \cdot \alpha N \cdot \rho D_F \cdot \rho D_F$  
Таким образом вычислительная сложность еще падает в $\rho^2$ раз

Top-1 accuracy on ImageNet: **70.6%**

## DenseNet [2016-2018, CVPE]

[Densely Connected Convolutional Networks](https://arxiv.org/abs/1608.06993)

![DenseNet структура](images/dense_structure.png)

### Ключевые особенности

1. Выход каждого слоя является входом каждого последующего слоя
1. Решают vanishing-gradient проблему
1. Усиливают feature propagation и переиспользование признаков
1. Уменьшают кол-во параметров

### Дизайн

![Архитектура DenseNet](images/dense_architecture.png)

1. Композитная функция $H_l$: $x_l = H_l([x_0, x_1, ..., x_{l-1}])$
1. Конкатенация $[x_0, x_1, ..., x_{l-1}]$ обеспечивается за счет использования Dense блоков, в которых размерности выходных карт одинаковы
1. Transition layers - используются между Dense блоками и выполняют функцию уменьшения пространственной и глубинной размерностей
1. Коэффициент роста $k$ - указывает кол-во карт признаков, которые добавляются очередным слоем. Итого, слой $l$ имеет $k_0 + k(l-1)$ входных карт признаков

### Результаты

Top-1 accuracy on ImageNet: **79.2%**  
Top-5 accuracy on ImageNet: **94.7%**

Уменьшение кол-ва параметров (объема вычислений):
201-слойный DenseNet с 20 млн параметров дает такую же точность классификации как 101-слойный ResNet с 40 млн параметров. Поэтому можно считать, что компрессия **в 2 раза**

## ResNeXt [2016-2017, CVPR]  
[Aggregated Residual Transformations for Deep Neural Networks](https://arxiv.org/abs/1611.05431)  
Реализация: [https://github.com/facebookresearch/ResNeXt](https://github.com/facebookresearch/ResNeXt)

Сравнение блоков ResNet и ResNetX: базовый блок разбивается на Cardinality=32 части, выходы с каждой части суммируются, к результату добавляется вход для формирования residual block.

![](./images/resnet_resnext_block.png)

Представленный ResNeXt-блок (картинка (a) ниже) сам по себе может быть сведен с помощью тензорных преобразований к блоку (b), формируя топологию, похожую на топологию Inception с той лишь разницей, что базовые блоки здесь представляют одну и ту же функцию.  
При этом блок (b) также может быть сведен к блоку (c), который является ни чем иным как групповой сверткой.  
Таким образом, ResNeXt-блок - это групповая свертка с residual-членом в виде входа.

![](images/resnext_equivalents.png)

**Преимущество блока ResNeXt перед Inception блоком**: используется один и тот же базовый блок, что приводит к меньшим усилиям при конфигурации сети.  
**Преимущество блока ResNeXt перед ResNet блоком**: сохраняя концепцию residual function и количество параметров в блоке, точность модели целиком становится выше.

![](./images/resnet_vs_resnext.png)

![](./images/resnet_resnext_train_curves.png)

Лучшее качество (при __Cardinality=64__ - количество групп, __d=4__ - количество входных == количество выходных каналов в базовой операции):  
Top-1 accuracy on ImageNet: **79.6%**  
Top-5 accuracy on ImageNet: **94.7%**